In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os

def train_and_serve_model(data_path, model_path="model.h5"):
    """
    Train a neural network on the NUSW-NB15 dataset and save the model for serving.

    Args:
        data_path (str): Path to the CSV dataset.
        model_path (str): Path to save the trained model (default: "model.h5").
    Returns:
        str: Message indicating training completion and model saving.
    """
    # Load dataset
    data = pd.read_csv(data_path)
    
    # Debug: Check column names
    print("Columns in the dataset:", data.columns)
    
    # Determine the target column
    target_column = 'attack_cat'  # Update this to match your dataset's target column
    
    # Encode categorical features
    categorical_cols = data.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if col != target_column:  # Don't encode the target column here
            data[col] = LabelEncoder().fit_transform(data[col])
    
    # Split into features (X) and target (y)
    X = data.drop(columns=[target_column, 'label'], axis=1)
    y = data[target_column]
    
    # Normalize features
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    # Encode target labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)
    
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.3, random_state=42)
    
    # Build the neural network
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(y_categorical.shape[1], activation='softmax')  # Output layer
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)
    
    # Save the trained model
    model.save(model_path)
    
    # Save the scaler and label encoder for consistent preprocessing
    np.save("scaler.npy", scaler)
    np.save("label_encoder.npy", label_encoder.classes_)
    
    return f"Model trained and saved to {model_path}"


def predict(input_data, model_path="model.h5"):
    """
    Predict the class of new input data using the trained model.

    Args:
        input_data (list): List of input features (single instance).
        model_path (str): Path to the trained model (default: "model.h5").
    Returns:
        str: Predicted class label.
    """
    # Load the trained model
    model = load_model(model_path)
    
    # Load the scaler and label encoder
    scaler = MinMaxScaler()
    scaler = scaler.fit(np.load("scaler.npy", allow_pickle=True))
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load("label_encoder.npy", allow_pickle=True)
    
    # Preprocess input data
    input_data = scaler.transform([input_data])
    
    # Predict the class
    predictions = model.predict(input_data)
    predicted_class = label_encoder.inverse_transform([np.argmax(predictions)])
    
    return predicted_class[0]


In [7]:
train_and_serve_model("./UNSW_NB15_testing-set.csv")

Columns in the dataset: Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')
Epoch 1/50


/home/mayank/anaconda3/envs/tensor/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-30 11:45:52.767260: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-30 11:45:52.959933: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and 

1441/1441 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7149 - loss: 0.9265 - val_accuracy: 0.8377 - val_loss: 0.4369
Epoch 2/50
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8434 - loss: 0.4401 - val_accuracy: 0.8576 - val_loss: 0.3947
Epoch 3/50
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8557 - loss: 0.4053 - val_accuracy: 0.8602 - val_loss: 0.3759
Epoch 4/50
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8580 - loss: 0.3891 - val_accuracy: 0.8648 - val_loss: 0.3619
Epoch 5/50
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8639 - loss: 0.3762 - val_accuracy: 0.8644 - val_loss: 0.3541
Epoch 6/50
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8667 - loss: 0.3630 - val_accuracy: 0.8607 - val_loss: 0.3518
Epoch 7/50
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8659 - loss: 0.3669 - val_accuracy: 0.8681 - val_loss: 0.3464
Epoch 8/50
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8705 - loss: 0.3484 - val_accurac

KeyboardInterrupt: 